In [61]:
import sys
sys.path.append('../')
from src import load_dataset

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, strip_accents_unicode
from nltk.stem.snowball import FrenchStemmer
from wordcloud import WordCloud

In [10]:
X, y = load_dataset()

In [127]:
stemmer = FrenchStemmer()
stopwords_list = stopwords.words('french')[:66] #on cut avant les différentes formes du verbe être/avoir qui seront handle par le stemmer

baseVectorizer = CountVectorizer(strip_accents="ascii", stop_words=stopwords_list)
preprocess = baseVectorizer.build_preprocessor()
analyzer = baseVectorizer.build_analyzer()
def mixed_analyzer(doc):
    """
    Stem avec nltk + regarde les stopwords
    """
    preprocess_doc = preprocess(doc)
    tokenized_list = analyzer(preprocess_doc)
    return [stemmer.stem(w) for w in tokenized_list]

/home/charles/M1-S2-DAC/RITAL/TAL/Projet/RITAL-Projet1/.venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['meme'] not in stop_words.
  warnings.warn(


In [128]:
vectorizer = CountVectorizer(
    analyzer=mixed_analyzer,
    )
X_transformed = vectorizer.fit_transform(X)

In [129]:
words = vectorizer.get_feature_names_out()
words[400:500]

array(['absenc', 'absent', 'absente', 'absolu', 'absolus', 'absolut',
       'absorb', 'absorpt', 'absten', 'abstent', 'abstention', 'abstenu',
       'abstract', 'abstrair', 'abstrait', 'absurd', 'absurdit', 'abuj',
       'abus', 'abym', 'acad', 'academ', 'academicien', 'acadien',
       'acapulco', 'accabl', 'accablent', 'accapar', 'acce', 'acced',
       'accedent', 'accederent', 'accel', 'acceler', 'accelere',
       'accelerent', 'accent', 'accentu', 'accentue', 'accentuent',
       'accept', 'accepte', 'acceptent', 'accepton', 'access',
       'accessibilit', 'accessibl', 'accessoir', 'accident', 'accidentel',
       'acclam', 'acclimat', 'accol', 'accommod', 'accommodent',
       'accomod', 'accompagn', 'accompagne', 'accompagnent',
       'accompagnon', 'accompl', 'accord', 'accorde', 'accordent',
       'accordon', 'accost', 'accouch', 'accourt', 'accredit', 'accroch',
       'accrochag', 'accroche', 'accroiss', 'accroissement',
       'accroissent', 'accroit', 'accroitr', 'a